In [39]:
import time
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore')

In [81]:
# Sleep function 
def sleep(x):
    time.sleep(x)

# Wait for a certain measure of time before throwing an exception
def wait(x):
    driver.implicitly_wait(x)

# Click Function
def click_bann_byID(ID):
    actions = ActionChains(driver)
    accept = driver.find_element(By.ID, ID)
    actions.click(accept).perform()
    wait(10)
    sleep(0.5)

    
# Find Elements Function
def find_elements_TAPF(T,A,P,F):

    titles = driver.find_elements(By.CLASS_NAME, T)
    address = driver.find_elements(By.CLASS_NAME, A)
    prices = driver.find_elements(By.CLASS_NAME, P)
    features = driver.find_elements(By.CLASS_NAME, F) 

    list_title = [title.text for title in titles]
    list_addres = [addr.text for addr in address]
    list_price = [price.text for price in prices]
    list_features = [feature.text for feature in features]
    return list_title, list_addres, list_price, list_features


'/html/body/div[3]/div[3]/div[3]/div/ul/li[3]/section/div[3]'
"listing-search-item__sub-title'"

"listing-search-item__sub-title'"

In [130]:
print('---------------------- Selenium Project Web-Scraping Project ----------------------')
start=datetime.now()

#  1 - Create Driver
Path = '/Users/macbook/Desktop/projects/Github_Repositories/Portfolio Projects/02 - Web_Scraping_Job_Search/chromedriver'
driver = webdriver.Chrome(Path)

#  2 - Go to Website
link = 'https://www.pararius.com/apartments/rotterdam?ac=1'

driver.get(link)
wait(10)
sleep(2)

# 3 - number of results
total_house_number = driver.find_elements(By.CLASS_NAME, 'search-list-header__count')
list_result = [tt.text for tt in total_house_number]
print(f' Total Number of house = {int(list_result[0])}\n' )


# Number of Page
#num_of_page = driver.find_elements(By.CLASS_NAME, '')

# 4 -  Take Infos from Page
# 4.1 - Headers, Publish_Time ,Company, City
T, A, P, F = 'listing-search-item__title', "listing-search-item__sub-title\\'", 'listing-search-item__price', 'listing-search-item__features'
list_title, list_addres, list_price, list_features = find_elements_TAPF(T,A,P,F)

print(f'{list_title[1]} , {list_addres[1]} , {list_price[1]} , {list_features[1]}\n')

all_list = [list_title, list_addres, list_price, list_features]

for i in all_list:
    print(len(i))

    
# 4.6 - DataFrame df
d = dict(title=np.array(list_title), address=np.array(list_addres), price=np.array(list_price),
         duzenle=np.array(list_features) )
df = pd.DataFrame.from_dict(d, orient='index')
df = df.T
    
# Page number deneme
page_num = driver.find_elements(By.CLASS_NAME, 'pagination')
list_page = [i.text for i in page_num]
a = str(list_page)
page_number = int(a[-10:-8])
print('Total Page number = ', page_number)

num = 1
while num < 3:
    try:
        num+=1

        # 4.7.1 - Go to another page
        'https://www.pararius.com/apartments/rotterdam/page-2'
        link = f'https://www.pararius.com/apartments/rotterdam/page-{page_number}'
        driver.get(link)
        wait(10)
        sleep(1.5)

        # Take infos from page
        list_title, list_addres, list_price, list_features = find_elements_TAPF(T,A,P,F)

        # Data Frame
        d = dict(title=np.array(list_title), address=np.array(list_addres), price=np.array(list_price),
             duzenle=np.array(list_features) )
        df2 = pd.DataFrame.from_dict(d, orient='index')
        df2 = df2.T
    except:
        pass

    # 4.7.4 - Concatenate the DataFrames
    df = pd.concat([df,df2], axis=0, ignore_index=True)
    print(f'Page Number : {num}, DataFrame Shape : {df2.shape}')
    

# Data Cleaning
df.duzenle = df.duzenle.str.replace('\n', '/')
df[['m2', 'rooms', 'furnished']] = df.duzenle.str.split('/', expand=True)
    
# 5.1 - Save Data as csv 
print(f'DataFrame End : {df.shape}')
df.to_csv('rotterdam.csv', index=False)


# 6 - Quit
end =datetime.now() 
print('Code Runned, No Problem')
print(f'Time = {end - start}')
sleep(5)
driver.quit()

df.head()

---------------------- Selenium Project Web-Scraping Project ----------------------
 Total Number of house = 617

House Burgemeester Meineszlaan , 3022 BC Rotterdam (Nieuwe Westen) , €2,800 per month , 82 m²
5 rooms

32
32
32
32
Page number =  21
Page Number : 2, DataFrame Shape : (19, 4)
Page Number : 3, DataFrame Shape : (19, 4)
DataFrame End : (70, 7)
Code Runned, No Problem
Time = 0:00:20.933129


,title,address,price,duzenle,m2,rooms,furnished
0,Apartment Stationssingel,3033 HE Rotterdam (Provenierswijk),"€1,595 per month",99 m²/3 rooms/Upholstered,99 m²,3 rooms,Upholstered
1,House Burgemeester Meineszlaan,3022 BC Rotterdam (Nieuwe Westen),"€2,800 per month",82 m²/5 rooms,82 m²,5 rooms,None
2,Apartment Boompjes,3011 XZ Rotterdam (Stadsdriehoek),"€1,625 per month",59 m²/2 rooms/Furnished,59 m²,2 rooms,Furnished
3,Apartment Hermonlaan 64,3061 BH Rotterdam (Kralingen West),"€2,000 per month",80 m²/3 rooms/Furnished,80 m²,3 rooms,Furnished
4,Apartment Hermonlaan 35,3061 BH Rotterdam (Kralingen West),"€1,495 per month",75 m²/3 rooms/Upholstered,75 m²,3 rooms,Upholstered


In [108]:
df = pd.read_csv('rotterdam.csv')
df.tail()

,title,address,price,duzenle
110,Apartment Bergselaan,3038 CT Rotterdam (Bergpolder),"€1,395 per month",72 m²\n4 rooms\nUpholstered or furnished
111,Apartment Maaskade 28 D,3071 NB Rotterdam (Noordereiland),"€1,450 per month",70 m²\n3 rooms\nUpholstered
112,Apartment Schiedamseweg,3026 AG Rotterdam (Tussendijken),"€1,600 per month",50 m²\n3 rooms\nFurnished
113,Apartment Wierdsmaplein,3072 MJ Rotterdam (Kop van Zuid),"€5,500 per month",170 m²\n3 rooms\nFurnished
114,Apartment Henegouwerlaan 43 C1,3021 CR Rotterdam (Middelland),"€1,450 per month",62 m²\n3 rooms\nUpholstered or furnished


In [112]:
#df[['lastname', 'firstname', 'middle initial']] = df['name'].str.split(expand=True)
df.duzenle = df.duzenle.str.replace('\n', '/')
df[['m2', 'rooms', 'furnished']] = df.duzenle.str.split('/', expand=True)
df.head()

,title,address,price,duzenle,m2,rooms,furnished
0,Apartment Stationssingel,3033 HE Rotterdam (Provenierswijk),"€1,595 per month",99 m²/3 rooms/Upholstered,99 m²,3 rooms,Upholstered
1,House Burgemeester Meineszlaan,3022 BC Rotterdam (Nieuwe Westen),"€2,800 per month",82 m²/5 rooms,82 m²,5 rooms,None
2,Apartment Hermonlaan 64,3061 BH Rotterdam (Kralingen West),"€2,000 per month",80 m²/3 rooms/Furnished,80 m²,3 rooms,Furnished
3,Apartment Hermonlaan 35,3061 BH Rotterdam (Kralingen West),"€1,495 per month",75 m²/3 rooms/Upholstered,75 m²,3 rooms,Upholstered
4,Apartment Willem de Zwijgerstraat 27 A,3043 VB Rotterdam (Overschie),"€1,100 per month",75 m²/3 rooms/1929,75 m²,3 rooms,1929


In [114]:
df.duplicated().sum()

129

In [119]:
list_page

['1\n2\n3\n4\n5\n...\n21\nNext']

In [125]:
a = str(list_page)
a

"['1\\n2\\n3\\n4\\n5\\n...\\n21\\nNext']"

In [127]:
int(a[-10:-8])

21